<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Encrypted_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1jjLla7K06juiqgEhoJdSW2d0Tz1eke7I#scrollTo=ZyWtMDj22Sqs

In [1]:
! pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 460kB 40.8MB/s 
     |████████████████████████████████| 204kB 41.1MB/s 
     |████████████████████████████████| 389kB 43.3MB/s 
     |████████████████████████████████| 81kB 22.1MB/s 
     |████████████████████████████████| 1.4MB 43.5MB/s 
     |████████████████████████████████| 256kB 28.1MB/s 
     |████████████████████████████████| 266kB 43.5MB/s 
     |████████████████████████████████| 51kB 18.8MB/s 
     |████████████████████████████████| 122kB 43.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067103 sha256=35ca79db56e2e7ba8498cfcaece29124e3c70d8073032e8bb3607d69e8414321
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=477e20aa508775704490773d8a33ebda1716afb712d6f93264b8075a37102eca
  Stored in di

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

import random
import numpy as np

W0808 08:04:20.579727 140127071491968 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0808 08:04:20.598074 140127071491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
import string

In [0]:
char2index = {}
index2char = {}

In [0]:
for i,char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
    char2index[char] = i
    index2char[i] = char

In [0]:
str_input = "Hello"
max_len = 8

In [0]:
def string2values(str_input, max_len=8):

    str_input = str_input[:max_len].lower()



    if(len(str_input) < max_len):
        str_input = str_input + "." * (max_len - len(str_input))

    values = list()
    for char in str_input:
        values.append(char2index[char])

    return th.tensor(values).long()

def values2string(input_values):
    s = ""
    for value in input_values:
        s += index2char[int(value)]
    return s

def strings_equal(str_a, str_b):

    vect = (str_a * str_b).sum(1)

    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x = x * vect[i + 1]    

    return x

def one_hot(index, length):
    vect = th.zeros(length).long()
    vect[index] = 1
    return vect

def string2one_hot_matrix(str_input, max_len=8):

    str_input = str_input[:max_len].lower()

    # pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + "." * (max_len - len(str_input))

    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2index[char], len(char2index)).unsqueeze(0)
        char_vectors.append(char_v)
        

In [0]:
class EncryptedDB():
    
    def __init__(self, *owners, max_key_len=8, max_val_len=8):
        self.max_key_len = max_key_len
        self.max_val_len = max_val_len
        
        self.keys = list()
        self.values = list()
        self.owners = owners
        
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2values(value, max_len=self.max_val_len)
        value = value.share(*self.owners)
        self.values.append(value)
        
    def query(self, query_str):
        query_matrix = string2one_hot_matrix(query_str)
        
        query_matrix = query_matrix.share(*self.owners)

        key_matches = list()
        for key in self.keys:

            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)

        result = self.values[0] * key_matches[0]

        for i in range(len(self.values) - 1):
            result += self.values[i+1] * key_matches[i+1]
            
        result = result.get()

        return values2string(result).replace(".","")

In [0]:
db = EncryptedDB()

In [17]:
db.keys

[]

In [29]:
db = EncryptedDB(bob, alice, secure_worker)
db.add_entry('ke1','value1')
db.add_entry('key2', 'value2')
db.add_entry('key3','value3')
db.add_entry('Key4','value4')


NameError: ignored

In [18]:
db.query('key1')

AttributeError: ignored

In [28]:
db.add_entry('ke1','value1')
db.add_entry('key2', 'value2')
db.add_entry('key3','value3')
db.add_entry('Key4','value4')


AttributeError: ignored

In [26]:
#db = EncryptedDB('bob', 'alice', secure_worker, max_val_len=256)

db.add_entry('bob',"(123) 456 7890")
db.add_entry("alice", "(234) 567 8901")
db.add_entry("Sam","(345) 678 9012")
db.add_entry("Key","really big json value")

db.query('bob')

AttributeError: ignored